# Autoencoder Simple
from https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/simple_autoencoder.py

In [1]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [3]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [4]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [5]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [10]:
# dataset = MNIST('./data', transform=img_transform)
# dataset = MNIST('D:/datasets_pytorch/MNIST30', download=True, transform=img_transform)
dataset = MNIST('D:/datasets_pytorch/MNIST30', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [26]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 4))
        self.decoder = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [29]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)
print(model)

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=12, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=12, out_features=4, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
)


In [28]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
#     print("type {}".format(type(loss.data)))
#     print("size {}".format(loss.data.size()))
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data))
    
#     print('epoch [{}/{}], loss:{:.4f}'
#           .format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.2129
epoch [2/100], loss:0.1647
epoch [3/100], loss:0.1581
epoch [4/100], loss:0.1367
epoch [5/100], loss:0.1391
epoch [6/100], loss:0.1369
epoch [7/100], loss:0.1133
epoch [8/100], loss:0.1351
epoch [9/100], loss:0.1263
epoch [10/100], loss:0.1258
epoch [11/100], loss:0.1424
epoch [12/100], loss:0.1342
epoch [13/100], loss:0.1143
epoch [14/100], loss:0.1344
epoch [15/100], loss:0.1121
epoch [16/100], loss:0.1328
epoch [17/100], loss:0.1173
epoch [18/100], loss:0.1187
epoch [19/100], loss:0.1184
epoch [20/100], loss:0.1248
epoch [21/100], loss:0.1185
epoch [22/100], loss:0.1190
epoch [23/100], loss:0.1022
epoch [24/100], loss:0.1189
epoch [25/100], loss:0.1180
epoch [26/100], loss:0.1102
epoch [27/100], loss:0.1139
epoch [28/100], loss:0.1085
epoch [29/100], loss:0.1095
epoch [30/100], loss:0.1196
epoch [31/100], loss:0.1125
epoch [32/100], loss:0.1221
epoch [33/100], loss:0.1155
epoch [34/100], loss:0.1145
epoch [35/100], loss:0.1115
epoch [36/100], loss:0.1087
e